In [4]:
#Importing spark session
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [6]:
spark = SparkSession\
.builder\
.appName("sparksql")\
.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/12 02:41:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
print(spark.version)

3.5.1


In [10]:
data = spark.read.format('csv')\
.option('inferSchema','true')\
.option('header','true')\
.option('path','operations_management.csv')\
.load()

In [12]:
data.printSchema()

root
 |-- description: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- level: integer (nullable = true)
 |-- size: string (nullable = true)
 |-- line_code: string (nullable = true)
 |-- value: integer (nullable = true)



In [14]:
#Using dataFrame API and it's transfomations
data_2 = data.select("industry","value")\
.filter((col("value") > 200) & (col("industry") != "total"))\
.orderBy(desc("value"))

In [16]:
data_2.printSchema()

root
 |-- industry: string (nullable = true)
 |-- value: integer (nullable = true)



In [18]:
data_2.show(10)

+--------------------+-----+
|            industry|value|
+--------------------+-----+
|        Construction| 6030|
|        Construction| 5904|
|        Construction| 5229|
|Accommodation & f...| 5058|
|        Construction| 4965|
|        Construction| 4959|
|Accommodation & f...| 4950|
|        Construction| 4686|
|        Construction| 4668|
|        Construction| 4665|
+--------------------+-----+
only showing top 10 rows



In [20]:
#Using Spark SQL funcation to submit SQL query, by creating a temporary view on top data
data_2.createOrReplaceTempView("data")

In [22]:
spark.sql("""SELECT industry,value 
FROM data 
WHERE 
value > 200 AND industry != "total"
""").show(5)

+--------------------+-----+
|            industry|value|
+--------------------+-----+
|        Construction| 6030|
|        Construction| 5904|
|        Construction| 5229|
|Accommodation & f...| 5058|
|        Construction| 4965|
+--------------------+-----+
only showing top 5 rows



In [30]:
data.createOrReplaceGlobalTempView("test")

In [32]:
data2 = spark.sql("""
Select * from test
""")

In [34]:
data2.show()

+--------------------+--------------------+-----+---------------+---------+-----+
|         description|            industry|level|           size|line_code|value|
+--------------------+--------------------+-----+---------------+---------+-----+
|Awareness of clim...|               total|    0| 6–19 employees| C0300.01|13080|
|Awareness of clim...|               total|    0|20–49 employees| C0300.01| 3348|
|Awareness of clim...|               total|    0|50–99 employees| C0300.01| 1089|
|Awareness of clim...|               total|    0| 100+ employees| C0300.01| 1023|
|Awareness of clim...|Agriculture, fore...|    1|          total| C0300.01| 2364|
|Awareness of clim...|         Agriculture|    2|          total| C0300.01| 1683|
|Awareness of clim...|  Commercial fishing|    2|          total| C0300.01|   27|
|Awareness of clim...|  Forestry & logging|    2|          total| C0300.01|  126|
|Awareness of clim...|Agriculture, fore...|    2|          total| C0300.01|  528|
|Awareness of cl

In [36]:
spark.catalog.listDatabases()

[Database(name='default', catalog='spark_catalog', description='default database', locationUri='file:/Users/ikramkhan/Apache%2520Spark/spark-warehouse')]

In [38]:
spark.catalog.listTables()

[Table(name='data', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='test', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [42]:
spark.catalog.dropGlobalTempView("test")

True